In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import numpy as np
import time
import random
import array
from numpy import array

# Set hyperparameters
learning_rate = .08
training_epochs = 100
display_steps = 10

# Network parameters
n_hidden = 100 # number of hidden nodes
n_output = 1 # number of unique outputs

n_input = 100
file_len = 4000; 
ninetyperc = int(file_len * .9)


In [43]:
def createData():
    lineArray=[]
    for i in range(41):
        with open('incidence/10_10000_5_sir_01_' + str(i) + '_inc.csv') as f:
            for line in f:
                lineArray.append(line.split())            
    return lineArray

def cleanData(data):
    for j in range(len(data)):
        for i in range(len(data[0])):
            e = data[j][i].replace(",", '')
            data[j][i] = e
    

def createX(data):
    newAr = [i[4:] for i in data]
    return newAr
def createY(data):
    newAr = [i[0] for i in data]
    return newAr

data = createData()
cleanData(data)
random.shuffle(data)

train_data = data[:ninetyperc]
test_data = data[ninetyperc:]


x_train = createX(train_data)
x_test = createX(test_data)
y_train = createY(train_data)
y_test = createY(test_data)

x_train = array(x_train) 
x_train = x_train.reshape(x_train.shape[0], n_input)
x_train.shape

x_test = array(x_test)
x_test = x_test.reshape(x_test.shape[0], n_input)

y_train = array(y_train)
y_train = y_train.reshape(y_train.shape[0], n_output)


y_test = array(y_test)
y_test = y_test.reshape(y_test.shape[0], n_output)

y_train.shape



(3600, 1)

In [44]:
# Graph nodes - TensorFlow
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])


# Weights and biases
weights = {
    "hidden" : tf.Variable(tf.random_normal([n_input, n_hidden]), name="weight_hidden"),
    "hidden2" : tf.Variable(tf.random_normal([n_hidden, n_hidden]), name="weight_hidden2"),
    "hidden3" : tf.Variable(tf.random_normal([n_hidden, n_hidden]), name="weight_hidden3"),
    "hidden4" : tf.Variable(tf.random_normal([n_hidden, n_hidden]), name="weight_hidden4"),
    "output" : tf.Variable(tf.random_normal([n_hidden, n_output]), name="weight_output")
}

bias = {
    "hidden" : tf.Variable(tf.random_normal([n_hidden]), name="bias_hidden"),
    "hidden2" : tf.Variable(tf.random_normal([n_hidden]), name="bias_hidden2"),
    "hidden3" : tf.Variable(tf.random_normal([n_hidden]), name="bias_hidden3"),
    "hidden4" : tf.Variable(tf.random_normal([n_hidden]), name="bias_hidden4"),
    "output" : tf.Variable(tf.random_normal([n_output]), name="bias_output")
}    
    
# Define the multilayer perceptron (MLP) model
def model(X, weights, bias):
    layer_1 = tf.add(tf.matmul(X, weights["hidden"]), bias["hidden"])
    layer_1 = tf.nn.tanh(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights["hidden2"]), bias["hidden2"])
    layer_2 = tf.nn.relu(layer_2)
    
    layer_3 = tf.add(tf.matmul(layer_2, weights["hidden3"]), bias["hidden3"])
    layer_3 = tf.nn.sigmoid(layer_3)
    
    layer_4 = tf.add(tf.matmul(layer_3, weights["hidden4"]), bias["hidden4"])
    layer_4 = tf.nn.tanh(layer_4)
    
    output_layer = tf.matmul(layer_4, weights["output"]) + bias["output"]
    return output_layer


# Define model
pred = model(X, weights, bias)

# Define loss and optimizer
cost = tf.reduce_mean(tf.losses.mean_squared_error(Y,pred))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
# Initializing global variables
init = tf.global_variables_initializer()
start_time = time.time()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        _, c = sess.run([optimizer, cost], feed_dict={X: x_train, Y: y_train})
        if(epoch + 1) % display_steps == 0:
            print("Epoch: " , (epoch+1), " Cost: ", c)
    print("Optimization finished!")
    
    
    train_result = sess.run(pred, feed_dict={X: x_train})
    test_result = sess.run(pred, feed_dict={X: x_test})
    
    y_test = np.array(y_test,dtype=np.float32) #Converts y_test to float
    

end_time = time.time()

print("Completed in ", end_time - start_time, " seconds.")

count = 0
errorTotal = 0
total = 0
for val in test_result:
    #print("Estimated: ", test_result[count], "Actual: ", y_test[count], "Abs Dif: ", abs(test_result[count]-y_test[count]))
    errorTotal = errorTotal + abs(test_result[count]-y_test[count])
    count = count + 1
    
print("Total Error: ", errorTotal, " Over ", count , " points")
print("Average error: ", errorTotal/count)

count = 0
for val in test_result:
    print("Estimated: ", test_result[count], "Actual: ", y_test[count], "Abs Dif: ", abs(test_result[count]-y_test[count]))
    count = count + 1
    



Epoch:  10  Cost:  9.672081
Epoch:  20  Cost:  2.6263661
Epoch:  30  Cost:  0.80817163
Epoch:  40  Cost:  0.8613505
Epoch:  50  Cost:  0.7610643
Epoch:  60  Cost:  0.62155795
Epoch:  70  Cost:  0.52955437
Epoch:  80  Cost:  0.5102991
Epoch:  90  Cost:  0.5005197
Epoch:  100  Cost:  0.49372417
Optimization finished!
Completed in  17.61304020881653  seconds.
Total Error:  [292.71265]  Over  500  points
Average error:  [0.5854253]
Estimated:  [3.0568373] Actual:  [2.545067] Abs Dif:  [0.51177025]
Estimated:  [3.0568416] Actual:  [3.73086] Abs Dif:  [0.6740184]
Estimated:  [2.8024538] Actual:  [3.5513115] Abs Dif:  [0.74885774]
Estimated:  [2.9753034] Actual:  [2.5872283] Abs Dif:  [0.3880751]
Estimated:  [2.371796] Actual:  [3.2769637] Abs Dif:  [0.9051678]
Estimated:  [2.4459221] Actual:  [3.7505922] Abs Dif:  [1.3046701]
Estimated:  [2.4852083] Actual:  [3.36891] Abs Dif:  [0.8837018]
Estimated:  [2.2357495] Actual:  [3.2424378] Abs Dif:  [1.0066884]
Estimated:  [2.4773872] Actual:  [1.

Estimated:  [3.015837] Actual:  [2.4723012] Abs Dif:  [0.5435357]
Estimated:  [2.6941283] Actual:  [2.5307338] Abs Dif:  [0.16339445]
Estimated:  [3.0640047] Actual:  [3.8722105] Abs Dif:  [0.80820584]
Estimated:  [3.0512931] Actual:  [2.8877485] Abs Dif:  [0.16354465]
Estimated:  [3.0520322] Actual:  [2.6277983] Abs Dif:  [0.4242339]
Estimated:  [2.5390399] Actual:  [2.170756] Abs Dif:  [0.36828375]
Estimated:  [2.5286663] Actual:  [1.7175633] Abs Dif:  [0.811103]
Estimated:  [2.6277788] Actual:  [2.8031957] Abs Dif:  [0.17541695]
Estimated:  [3.051242] Actual:  [3.470381] Abs Dif:  [0.4191389]
Estimated:  [2.6562674] Actual:  [3.3755646] Abs Dif:  [0.7192972]
Estimated:  [2.9612997] Actual:  [3.1872091] Abs Dif:  [0.22590947]
Estimated:  [2.4858253] Actual:  [1.6044384] Abs Dif:  [0.8813869]
Estimated:  [2.9751189] Actual:  [2.8019695] Abs Dif:  [0.17314935]
Estimated:  [2.6542132] Actual:  [2.2485855] Abs Dif:  [0.40562773]
Estimated:  [2.6784666] Actual:  [3.0583] Abs Dif:  [0.3798

Estimated:  [2.3732631] Actual:  [2.777677] Abs Dif:  [0.40441394]
Estimated:  [2.373253] Actual:  [1.9919012] Abs Dif:  [0.38135195]
Estimated:  [2.3320467] Actual:  [3.148613] Abs Dif:  [0.8165662]
Estimated:  [3.0471747] Actual:  [3.2360232] Abs Dif:  [0.1888485]
Estimated:  [2.8890884] Actual:  [2.6024141] Abs Dif:  [0.28667426]
Estimated:  [2.3733342] Actual:  [2.6311958] Abs Dif:  [0.2578616]
Estimated:  [2.9994242] Actual:  [3.8216124] Abs Dif:  [0.82218814]
Estimated:  [2.6368887] Actual:  [3.264384] Abs Dif:  [0.6274953]
Estimated:  [3.0520322] Actual:  [3.7522366] Abs Dif:  [0.7002044]
Estimated:  [3.0520322] Actual:  [3.0595577] Abs Dif:  [0.00752544]
Estimated:  [2.3733656] Actual:  [2.520699] Abs Dif:  [0.14733338]
Estimated:  [2.9445007] Actual:  [2.2348833] Abs Dif:  [0.7096174]
Estimated:  [2.9612997] Actual:  [2.3929954] Abs Dif:  [0.5683043]
Estimated:  [2.9751189] Actual:  [2.5802968] Abs Dif:  [0.39482212]
Estimated:  [2.3744419] Actual:  [1.6282146] Abs Dif:  [0.74